# Final Assignment B

## Part 1: Data Preparation

This should not be exaggerated: We need to accomplish the following:
 - decide which columns to keep
 - decide which rows to keep
 - decide how to transmute certain features to increase information content

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import time

from matplotlib import rc
rc('font',**{'family':'serif','sans-serif':['Computer Modern Roman']})
## for Palatino and other serif fonts use:
#rc('font',**{'family':'serif','serif':['Palatino']})
rc('text', usetex=True)

In [ ]:
path = '../diabetes/diabetic_data_original.csv'

data = pd.read_csv(path)

In [ ]:
data.sample(6)

In [ ]:
data.info()

"""
oh dear... all the null values in this data set are just placeholders, like ? in weight....
"""

In [ ]:
(data.patient_nbr.value_counts() ).value_counts()/len(data.patient_nbr.value_counts())
# We will have to drop more than half of rows exclusively due to sample isolation...

In [ ]:
counts = data.patient_nbr.value_counts()
inds_kept = data.patient_nbr.map(lambda x: counts[x] > 1).values
reduced_data = data[inds_kept]

In [ ]:
reduced_data.time_in_hospital.value_counts() # Excellent, no missing labels.

In [ ]:
#(reduced_data.payer_code.value_counts())/len(reduced_data)
1-(reduced_data.patient_nbr.value_counts() > 1).value_counts()/len(reduced_data)

In [ ]:
def cat_diag(diag):
    if (type(diag) == float and np.isnan(diag)) or diag == "NaN" or diag == '?':
        return "NULL"
    try:
        num = int(diag)
    except Exception as e:
        if diag[0] == 'E':
            return "SUPP_CLASS_EXTERNAL_CAUSE"
        elif diag[0] == 'V':
            return "SUPP_CLASS_HEALTH_FACTORS"
        else:
            try:
                num = float(diag)
                return "DIABETES"
            except Exception as ex:
                return "ERROR"
    if (num >= 390 and num <= 459) or num == 785:
        return "CIRCULATORY"
    elif (num == 250):
        return "DIABETES"
    elif (num >= 460 and num <= 519) or num == 786:
        return "RESPIRATORY"
    elif (num >= 520 and num <= 579) or num == 787:
        return "DIGESTIVE"
    elif num >= 800: 
        return "INJURY_OR_POISON"
    elif num >= 710 and num <= 739:
        return "MUSKOSKELETAL"
    elif (num >= 580 and num <= 629) or num == 788:
        return "GENITOURINARY"
    elif num >= 140 and num <= 239:
        return "NEOPLASMS"
    elif num >= 680 and num <= 709:
        return "SKIN"
    elif num >= 780 and num <= 799:
        return "ILL-DEFINED"
    elif num <= 139:
        return "INFECTIOUS_PARASITIC"
    elif num >= 240 and num <= 279:
        return "ENDOCRINE_METABOLIC"
    elif num >= 290 and num <= 319:
        return "MENTAL_DISORDER"
    elif num >= 630 and num <= 679:
        return "PREGNANCY_COMPLICATIONS"
    elif num >= 280 and num <= 289:
        return "BLOOD"
    elif num >= 320 and num <= 389:
        return "NERVOUS_SENSE"
    elif num >= 740 and num <= 759:
        return "CONGENITAL"
    else: 
        return "ERROR"

In [ ]:
# Changes from assignment A: 
# - no "boolean" glucose serum test
# - not dropping rows w/ missing race data, but masking the missing values
# - no dropping of particular discharge dispositions
# - no dropping of payer code and medical specialty (just use masking)

def cat_ads(asid):
    if asid in [17,20,9]:
        return "NULL"
    elif asid in [1,2,3]:
        return "REFERRAL"
    elif asid in [4,5,6]:
        return "TRANSFER"
    elif asid in [7]:
        return "E.R."
    elif asid in [8]:
        return "LAW_ENFORCEMENT"
    else:
        return "OTHER"
    
def cat_adt(atid):
    cats = [0, "Emergency","Urgent","Elective","Newborn", "NULL", "NULL", "Trauma Center", "NULL"]
    return cats[atid]

def cat_dsch(did):
    if did in [18,25,26]:
        return "NULL"
    elif did in [3,4,5]:
        return "ICF/SNF"
    elif did in [6,8,12]:
        return "FURTHER_CARE_HOME"
    elif did in [7]:
        return "LEFT_AMA"
    elif did in [15,17,9]:
        return "CARE_CONTINUES_IN_THIS_HOSPITAL"
    elif did in [13,14]:
        return 'HOSPICE'
    elif did in [11, 19, 20, 21]:
        return 'EXPIRY'
    elif did in [1]:
        return 'WENT_HOME'
    else:
        return "CARE_CONTINUES_ELSEWHERE"

In [ ]:
for col in reduced_data.columns:
    print(col)
    counts = reduced_data[col].value_counts()
    if '?' in counts.index:
        print(counts['?'], f"({counts['?']/len(reduced_data)})")
    else:
        print('No missing values')
    print()

In [ ]:
reduced_data.A1Cresult.value_counts()

In [ ]:
def get_next_visit_duration(encounter_id):
    try:
        patient_id = reduced_data.loc[reduced_data.encounter_id == encounter_id].patient_nbr.values[0]
        encounters = reduced_data.loc[reduced_data.patient_nbr == patient_id]
        encounters = encounters.loc[encounters.encounter_id > encounter_id]
        return encounters['time_in_hospital'].values[0]
    except Exception:
        return float('NaN')

In [ ]:
reduced_data['next_encounter_duration'] = reduced_data.encounter_id.map(get_next_visit_duration)

In [ ]:
reduced_data.next_encounter_duration.value_counts().sum(), len(reduced_data)

In [ ]:
reduced_data['discharge_disposition_cat'] = reduced_data.discharge_disposition_id.map(cat_dsch)
reduced_data['discharge_disposition_id'] = reduced_data.discharge_disposition_id.map(str)

reduced_data.age = reduced_data.age.map(lambda x: int(x[1]))

reduced_data['admission_source_id'] = reduced_data.admission_source_id.map(int)
reduced_data['admission_source_cat'] = reduced_data.admission_source_id.map(cat_ads)
reduced_data['admission_source_id'] = reduced_data.admission_source_id.map(str)

reduced_data['admission_type_id'] = reduced_data.admission_type_id.map(int)
reduced_data['admission_type_cat'] = reduced_data.admission_type_id.map(cat_adt)
reduced_data['admission_type_id'] = reduced_data.admission_type_id.map(str)

generics = ['metformin', 'repaglinide', 'nateglinide', 'chlorpropamide', 'glimepiride', 'acetohexamide', 
            'glipizide', 'glyburide', 'tolbutamide', 'pioglitazone', 'rosiglitazone', 'acarbose', 
            'miglitol', 'troglitazone', 'tolazamide', 'examide', 'insulin', 'glyburide-metformin', 
            'glipizide-metformin', 'glimepiride-pioglitazone', 'metformin-rosiglitazone', 
            'metformin-pioglitazone']

reduced_data['UP'] = np.sum([reduced_data[g] == 'Up' for g in generics], axis=0)
reduced_data['STEADY'] = np.sum([reduced_data[g] == 'Steady' for g in generics], axis=0)
reduced_data['DOWN'] = np.sum([reduced_data[g] == 'Down' for g in generics], axis=0)
reduced_data['NO'] = np.sum([reduced_data[g] == 'No' for g in generics], axis=0)

#reduced_data['age'] = reduced_data.age.map(lambda a: int(a[1]))

reduced_data['diagnoses'] = list(map(list, reduced_data[['diag_1', 'diag_2', 'diag_3']].values))
reduced_data['diagnoses'] = reduced_data.diagnoses.map(lambda l:[cat_diag(i) for i in l if not (type(i)==float and np.isnan(i))])

reduced_data['total_visits'] = reduced_data.number_emergency + reduced_data.number_inpatient + reduced_data.number_outpatient

reduced_data.drop(['weight'], axis=1)
reduced_data.dropna(subset=['next_encounter_duration'], inplace=True)

reduced_data.info()

In [ ]:
one_hot_simple = ['race', 
                  'gender', 
                  'A1Cresult', 
                  'change', 
                  'diabetesMed',
                  'discharge_disposition_cat',
                  'admission_type_cat',
                  'admission_source_cat',
                  'medical_specialty',
                  'readmitted'
                 ] +generics

poly_hot = ['diagnoses']


# Encoding for numerical data:

norm_simple = ['age']

std_simple = ['time_in_hospital', 
                'num_lab_procedures', 
                'num_procedures', 
                'num_medications', 
                'total_visits'] # , 'UP', 'STEADY', 'DOWN']

In [ ]:
len(reduced_data.medical_specialty.value_counts().index)

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, LabelBinarizer
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from sklearn.pipeline import Pipeline, FeatureUnion

from sklearn.base import BaseEstimator, TransformerMixin 

class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
    def fit(self, X, y=None):
        return self
    def transform(self, X):
        return X[self.attribute_names].values

class LabelBinarizerDitchNull(TransformerMixin):
    def __init__(self, num_features, *args, **kwargs):
        self.n = num_features
        self.encoders = []
        for _ in range(self.n):
            self.encoders.append(LabelBinarizer(*args, **kwargs))
    def fit(self, X, y=0):
        for i in range(self.n):
            self.encoders[i].fit(X[:, i])
        return self
    def transform(self, X, y=0):
        samples, n = X.shape
        assert n == self.n
        reses = []
        for i, encoder in enumerate(self.encoders):
            resi = encoder.transform(X[:, i])
            if "XXXXXXXXXXX" in encoder.classes_:
                inds = encoder.classes_ != "NULL"
                resi = resi[:, inds]
            reses.append(resi)
        num_new_features = sum([len(Z.T) for Z in reses])
        res = np.zeros((samples, num_new_features))
        i = 0
        for Z in reses:
            m, k = Z.shape
            assert m == samples
            res[:, i:i+k] = Z
            i += k
        return res
    
class PolyBinarizer(TransformerMixin):
    def __init__(self, num_features, *args, **kwargs):
        self.n = num_features
        self.encoders = []
        for _ in range(self.n):
            self.encoders.append(dict())
    def fit(self, X, y=0):
        transforms = []
        for i, encoder in enumerate(self.encoders):
            encoder['cats'] = sorted(np.unique(np.sum(X[:, i])))
            def itransform(X):
                samples = len(X)
                res = np.zeros((samples, len(encoder['cats'])))
                for j,x in enumerate(X):
                    for cat in x:
                        ind = encoder['cats'].index(cat)
                        res[j, ind] += 1
                return res
            transforms.append(lambda X : itransform(X))
        for i in range(self.n):
            self.encoders[i]['transform'] = transforms[i]
        return self
    def transform(self, X, y=0):
        samples, n = X.shape
        assert n == self.n
        reses = []
        for i, encoder in enumerate(self.encoders):
            resi = encoder['transform'](X[:, i])
            reses.append(resi)
        num_new_features = sum([len(Z.T) for Z in reses])
        res = np.zeros((samples, num_new_features))
        i = 0
        for Z in reses:
            m, k = Z.shape
            res[:, i:i+k] = Z
            i += k
        return res

In [ ]:
oh_pipe_ditch_null = Pipeline([
    ('selector', DataFrameSelector(one_hot_simple)),
    ('label_binarizer', LabelBinarizerDitchNull(num_features=len(one_hot_simple))),
])

poly_hot_pipe = Pipeline([
    ('selector', DataFrameSelector(poly_hot)),
    ('label_binarizer', PolyBinarizer(num_features = len(poly_hot)))
])

norm_pipeline = Pipeline([
    ('selector', DataFrameSelector(norm_simple)),
    ('scaler', MinMaxScaler())
])

std_pipeline = Pipeline([
    ('selector', DataFrameSelector(std_simple)),
    ('scaler', StandardScaler())
])

full_pipeline = FeatureUnion(transformer_list=[
        ("norm_pipeline", norm_pipeline),
        ("std_pipeline", std_pipeline),
        ("poly_hot_pipe", poly_hot_pipe), 
        ("oh_pipe_ditch_null", oh_pipe_ditch_null),
    ])

In [ ]:
reduced_data[generics]

In [ ]:
prepared_data = full_pipeline.fit_transform(reduced_data)
prepared_data.shape

In [ ]:
X = prepared_data
y = reduced_data.next_encounter_duration.values

X.min()

In [ ]:
X = X - np.min(X) + 1e-7
X.shape

## Part 2: Loss Function

This part should be very brief. We need to decide on a suitable 'loss funtion', that is, we need to decide on a probabilistic model we would like to model our data on, and we find out what function we would like to maximise wrt its parameters based on features $x$, and a label $y$.

The briefing text wsuggests the Poisson distribution, i.e. $Y_i \sim \mathrm{Poiss}(f_\theta(x_i))$. However, the support of the Poisson distribution is the set of natural numbers, we care about a finite support because of the way that `time_in_hospital` was measured (capping its value at 14 and giving it minimum value 1). Therefore, I suggest the model $Y_i - 1 \sim \mathrm{B}(13, (f_\theta(x_i)-1)/13)$ (a binomial distribution with 13 trials and expectation $f_\theta(x_i) - 1$), which will give $Y_i$ a support of $[14] = \{1, ... , 14\}$.

In order to model this, we will want to maximise the following:

$$
    \sum_i (y_i - 1) \log\left(f_\theta(x_i)-1\right) + (14-y_i)\log\left(14 - f_\theta(x_i)\right)
$$

with respect to the parameters $\theta$ of the models.

## Part 3: Machine Learning Algorithms Implementation

In this part, we actually build and train our neural nets. For this, we will need to define the loss function we derived above in Python, and choose an optimiser, in addition to playing around with the networks themselves.

In [ ]:
# Part 3.1: Simple neural nets...

import tensorflow.keras as keras
import tensorflow as tf
import pandas
import numpy as np
import matplotlib.pyplot as plt
import tensorflow.keras.backend as K

def lossfn (y, y_): 
    return -K.sum((y-1.0)*K.log(y_+K.epsilon()) + (14.0-y)*K.log(14-y_+K.epsilon()), axis=-1)

ytrue, ypred = tf.constant([1,6,1,4], dtype=float), tf.constant([2,3,2,3],dtype=float)

print(keras.losses.poisson(ytrue, ypred).numpy())
lossfn(ytrue, ypred).numpy()

In [ ]:
ins = keras.layers.Input(shape=(121,))
x = keras.layers.Dense(300)(ins)
x = keras.activations.tanh(x)
x = keras.layers.Dense(250)(x)
x = keras.layers.ReLU()(x)
x = keras.layers.Dense(100)(x)
x = keras.layers.ReLU()(x)
x_ = keras.layers.Dense(50)(ins)
x = keras.layers.ReLU()(x_)
x = keras.layers.Dense(25)(x)
x = keras.activations.exponential(x)
x = keras.layers.Dense(1)(x)
x = keras.layers.ReLU()(x)
#p = keras.layers.Dense(3)(x_)
#p = keras.activations.softmax(p)

model = keras.Model(inputs=ins, outputs=x)
model.compile(loss=lossfn, optimizer=keras.optimizers.Adam()) # keras.losses.mean_squared_error

In [ ]:
# These are some messy pieces of Python magic, to make interactive training of a neural network a little
# bit smoother. See the notes under Regression | Neural Network Model for how to use them.

import IPython
import signal

def interrupted(_interrupted=[False], _default=[None]):
    if _default[0] is None or signal.getsignal(signal.SIGINT) == _default[0]:
        _interrupted[0] = False
        def handle(signal, frame):
            if _interrupted[0] and _default[0] is not None:
                _default[0](signal, frame)
            print('Interrupt!')
            _interrupted[0] = True
        _default[0] = signal.signal(signal.SIGINT, handle)
    return _interrupted[0]

def enumerate_cycle(g):
    epoch = 0
    while True:
        for i,x in enumerate(g):
            yield (epoch,i), x
        epoch = epoch + 1

In [ ]:
# Set up an endless iteration through epochs and batches of data
BATCH_SIZE = 100
indexes = np.arange(len(X))
indexes = np.array_split(indexes, len(X) / BATCH_SIZE)
eb_indexes = enumerate_cycle(indexes)

# Accumulate some stats as we go along
loss_history = []

In [ ]:
while not interrupted():
    if(len(loss_history) > 0 and loss_history[-1][0]-loss_history[0][0] >= 300):
        break
    (epoch,batch),i = next(eb_indexes)
    #print(model.predict_on_batch(X[i]))
    #input()
    loss = model.train_on_batch(X[i], y1[i])
    if batch % 100 == 0:
        IPython.display.clear_output(wait=True)
        print(f'loss={loss} epoch={epoch} batchnum={batch}/{len(indexes)}')
        loss_history.append((time.time(), epoch+batch/len(indexes), loss))

In [ ]:
a = model._layers[-1]
a.__getstate__()

In [ ]:
#y2[i][:5]
print(keras.losses.binary_crossentropy(tf.constant(y2[i][:5]), (model.predict_on_batch(X[i][:5]))[-1]), '\n')
print(y2[i][:5], '\n') 
print(model.predict_on_batch(X[i][:5]), '\n')
print(np.array(loss_history)[-5:, -1], model.metrics_names)

In [ ]:
# Simple plot of the training loss
df = pandas.DataFrame.from_records(loss_history, columns=['time','epoch','loss'])
df.time = df.time - df.time[0]
df['loss1'] = df.loss.map(lambda x: x[1])
df['loss2'] = df.loss.map(lambda x: x[2])
print("Final training loss:", df.loss.ewm(com=5).mean().iloc[-1])
fig,ax = plt.subplots(figsize=(6,2))
#ax.plot(df.time, df.loss1)
ax.plot(df.time, df.loss1.ewm(com=5).mean(), linewidth=1)
ax.plot([0, 50],[-26.4, -26.4])
#ax.plot(df.time, df.loss2-3)
#ax.plot(df.time, df.loss2.ewm(com=10).mean()-3.6, linewidth=3)
ax.set_xlim([0,45])
plt.show()
df.time

In [ ]:
y[i][:5]

In [ ]:
from sklearn.decomposition import PCA

pca = PCA()

pca_res = pca.fit_transform(X)

X_ = (pca_res[:, :70])
X_ -= X_.min() - K.epsilon()

In [ ]:
insz = keras.layers.Input(shape=(70,))
z = keras.layers.Dense(100)(insz)
z = keras.activations.tanh(z)
z = keras.layers.Dense(75)(z)
z = keras.layers.ReLU()(z)
z = keras.layers.Dense(50)(z)
z = keras.layers.ReLU()(z)
z = keras.layers.Dense(25)(z)
z = keras.layers.ReLU()(z)
z = keras.layers.Dense(1)(z)
#x = keras.backend.argmax(x, dtype=float)

modelz = keras.Model(inputs=insz, outputs=z)
modelz.compile(loss=keras.losses.poisson, optimizer=keras.optimizers.Adam()) # keras.losses.mean_squared_error

In [ ]:
while not interrupted():
    if(len(loss_history) > 0 and loss_history[-1][0]-loss_history[0][0] >= 60):
        break
    (epoch,batch),i = next(eb_indexes)
    #print(model.predict_on_batch(X[i]))
    #input()
    loss = modelz.train_on_batch(X_[i], y[i])
    if batch % 100 == 0:
        IPython.display.clear_output(wait=True)
        print(f'loss={loss} epoch={epoch} batchnum={batch}/{len(indexes)}' + f' time={loss_history[-1][0]-loss_history[0][0]}' if epoch > 0 else '')
        loss_history.append((time.time(), epoch+batch/len(indexes), loss))

In [ ]:
modelz.predict_on_batch(X_[i][:5])

In [ ]:
y[i][:5]

In [ ]:
# Simple plot of the training loss
df = pandas.DataFrame.from_records(loss_history, columns=['time','epoch','loss'])
df.time = df.time - df.time[0]
print("Final training loss:", df.loss.ewm(com=5).mean().iloc[-1])
fig,ax = plt.subplots(figsize=(6,2))
ax.plot(df.time, df.loss)
ax.plot(df.time, df.loss.ewm(com=5).mean(), linewidth=3)
ax.set_ylim([-3.5,0])
plt.show()
df.time[-5:]

In [ ]:
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2, verbose=1, perplexity=50)
tsne_res = tsne.fit_transform(prepared_data)

In [ ]:
tx, ty = tsne_res[:, 0], tsne_res[:, 1]

plt.scatter(tx, ty)
plt.savefig('tsne.png')

In [ ]:
tsne2 = TSNE(n_components=2, verbose=1)
tsne2_res = tsne.fit_transform(X_)

In [ ]:

tx2, ty2 = tsne_res[:, 0], tsne_res[:, 1]
fig, ax=plt.subplots()
ax.scatter(tx2, ty2, s=0.2)
plt.show()

from sklearn.cluster import KMeans
np.random.seed(28)
km = KMeans(n_clusters=8)
km.fit(tsne_res)

fig, ax = plt.subplots(figsize=(5,5))

for inds, c, lab in zip([last_train, last_dev, last_val], ['#66c2a5', '#fc8d62', '#8da0cb'], ['Training', 'Development', 'Test']):
    ax.scatter(tx2[inds], ty2[inds], s=0.5, c=c, label=lab)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
if False:
    ax.spines['left'].set_color("white")
    ax.spines['bottom'].set_color("white")
else:
    ax.spines['left'].set_visible(False)
    ax.spines['bottom'].set_visible(False)
list(map(lambda ts: ts.set_color('white'),ax.xaxis.get_majorticklines()))
list(map(lambda ts: ts.set_color('white'),ax.yaxis.get_majorticklines()))
ax.set_aspect('equal')
fig.tight_layout()
fig.legend(frameon=False)    
plt.savefig('figures/datasplit.pdf')
    
np.array([True, True, False]) & np.array([False, True, False])

In [ ]:
dev = km.labels_ == 3
val = km.labels_ == 4
train = ~(dev | val)
D = X[dev]
V = X[val]
T = X[train]
ytrain = y[train]
ydev = y[dev]
yval = y[val]

In [ ]:
all((~last_train | train)), last_train ^ train, last_train, train

In [ ]:
for i,p in enumerate(patient_batches):
    if p[0].shape[0] != p[1].shape[0]:
        print(i)

In [ ]:
# Recurrent stuff:
# need to first separate data set into samples of patients. 

def get_patient(patient_nbr, samples=X, labels=y1):
    inds = reduced_data.patient_nbr == patient_nbr
    return samples[inds], labels[inds]

patient_batches = [get_patient(nbr) for nbr in np.unique(reduced_data.patient_nbr.values)]

ei_patients = enumerate_cycle(patient_batches)
loss_history_rec = []

In [ ]:
insrec = keras.layers.Input(shape=(None,121))
xrec = keras.layers.GRU(units=150, return_sequences=True)(insrec)
xrec = keras.layers.GRU(units=75, return_sequences=True)(xrec)
xrec = keras.layers.Dense(40)(xrec)
xrec = keras.layers.ReLU()(xrec)
xrec = keras.layers.Dense(1)(xrec)
modelrec = keras.models.Model(inputs=insrec, outputs=xrec)

def recloss(y, y_): return keras.losses.poisson(y[0,-1], y_[:,-1,0])

modelrec.compile(loss=lossfn, optimizer=keras.optimizers.Adam())

In [ ]:
ei_patients = enumerate_cycle(train_batches)
loss_history_rec = []

while not interrupted():
    if(len(loss_history_rec) > 0 and loss_history_rec[-1][0]-loss_history_rec[0][0] >= 300):
        break
    (epoch,item),patient = next(ei_patients)
    loss = modelrec.train_on_batch(patient[0][np.newaxis,:,:], patient[1][np.newaxis, :, np.newaxis])
    if item % 100 == 0:
        IPython.display.clear_output(wait=True)
        print(f'loss={loss} epoch={epoch} item={item}/{len(patient_batches)}' + ( f' time={loss_history_rec[-1][0]-loss_history_rec[0][0]}' if len(loss_history_rec) > 0 else ''))
        loss_history_rec.append((time.time(), epoch+item/len(patient_batches), loss))

In [ ]:
patient[0][np.newaxis,:,:]

In [ ]:
y__, y_t = modelrec.predict_on_batch(patient[0][np.newaxis,:,:]), patient[1][np.newaxis, :, np.newaxis]
#recloss(y_t, y__)
print(y_t.reshape(-1), y__.numpy().reshape(-1))

In [ ]:
a = [1,2,3]
a.extend([1,2,3])
a

In [ ]:
%matplotlib notebook

# Simple plot of the training loss
df = pandas.DataFrame.from_records(loss_history_rec, columns=['time','epoch','loss'])
df.time = df.time - df.time[0]
print("Final training loss:", df.loss.ewm(com=5).mean().iloc[-1])
fig,ax = plt.subplots(figsize=(6,2))
ax.plot(df.time, df.loss, alpha=0.2)
ax.plot(df.time, df.loss.ewm(com=5).mean(), linewidth=1)
ax.plot([0, 300], [-3,-3])
#ax.set_ylim([-5,5])
plt.show()
df.time[-5:]

In [ ]:
%matplotlib notebook

# Simple plot of the training loss
df = pandas.DataFrame.from_records(loss_history_rec, columns=['time','epoch','loss'])
df.time = df.time - df.time[0]
print("Final training loss:", df.loss.ewm(com=5).mean().iloc[-1])
fig,ax = plt.subplots(figsize=(6,2))
ax.plot(df.time, df.loss, alpha=0.2)
ax.plot(df.time, df.loss.ewm(com=5).mean(), linewidth=1)
ax.plot([0, 300], [-26.4,-26.4])
#ax.set_ylim([-5,5])
plt.show()
df.time[-5:]

In [ ]:
from tqdm import tqdm_notebook as tqdm

ys_ = []
ys = []
for p in tqdm(patient_batches):
    ys_.extend(modelrec.predict_on_batch(p[0][np.newaxis,:,:]).numpy().reshape(-1))
    ys.extend(p[1].reshape(-1))

fig, ax = plt.subplots()
ax.hist(ys_, bins=14*5)

In [ ]:
np.max(ys_)

In [ ]:
ysz_ = model.predict_on_batch(X)[0].numpy().reshape(-1)
fig, ax = plt.subplots()
ax.hist(y1, bins=14*2)
ax.hist(ysz_, bins=14*5)
plt.show()

In [ ]:
ysz_[0:10]

In [ ]:
patient_nbr = 88785891 #np.random.choice(reduced_data.patient_nbr.values)
X_p, yp = get_patient(patient_nbr, samples=X)

ypz_ = model.predict_on_batch(X_p)
ypr_ = modelrec.predict_on_batch(X_p[np.newaxis,:,:])


In [ ]:
ypz_[0], ypr_, yp

In [ ]:
fig, ax = plt.subplots()

ax.plot(ypz_[0].numpy().reshape(-1)[1:])
#ax.plot(ypr_.numpy().reshape(-1))
ax.plot(yp)

plt.show()

In [ ]:
model.predict_on_batch(X)

In [ ]:
reduced_data.patient_nbr.value_counts()

In [ ]:
ys_= val_rec_poiss

fig, ax = plt.subplots()

for (y, y_) in tqdm(zip (recyval, ys_)):
    if interrupted(): break
    ax.plot([y-0.5, y+0.5], [y_, y_], linewidth=0.5, c='black')

    ax.set_ylim([0.75, 14.25])
plt.show()

In [ ]:
len(recyval)

In [ ]:
ypr_.numpy().reshape(-1)

In [ ]:
len(ys_)

In [ ]:
inst = keras.layers.Input(shape=(121,))
t = keras.layers.Dense(100)(inst)
t = keras.activations.tanh(t)
t = keras.layers.Dense(75)(t)
t = keras.layers.ReLU()(t)
t = keras.layers.Dense(50)(t)
t = keras.layers.ReLU()(t)
t = keras.layers.Dense(50)(t)
t = keras.layers.ReLU()(t)
t = keras.layers.Dense(14)(t)
t = keras.activations.softmax(t)

modelt = keras.models.Model(inputs=inst, outputs=t)
modelt.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam())

In [ ]:
ycat = keras.utils.to_categorical(y1.reshape(-1,1))[:, 1:]
ycat.shape

In [ ]:
BATCH_SIZE = 100
indices = np.arange(len(X))
indices = np.array_split(indices, len(X) / BATCH_SIZE)
eb_indices = enumerate_cycle(indices)

# Accumulate some stats as we go along
loss_history = []

In [ ]:
while not interrupted():
    if(len(loss_history) > 0 and loss_history[-1][0]-loss_history[0][0] >= 300):
        break
    (epoch,batch),i = next(eb_indices)
    #print(model.predict_on_batch(X[i]))
    #input()
    loss = modelt.train_on_batch(X[i], ycat[i])
    if batch % 100 == 0:
        IPython.display.clear_output(wait=True)
        print(f'loss={loss} epoch={epoch} batchnum={batch}/{len(indices)}')
        loss_history.append((time.time(), epoch+batch/len(indices), loss))

In [ ]:
np.argmax(np.array([[1,4,5],[3,2,6]]), axis=1)

In [ ]:
ycat.sum(axis=0)

In [ ]:
yst_ = np.argmax(modelt.predict_on_batch(X).numpy(), axis=1)+1

In [ ]:
fig, ax= plt.subplots()
ax.hist(yst_)
plt.show()

In [ ]:
len(y1)

In [ ]:
modelt.predict_on_batch(X).numpy()[:, np.array(y1, dtype=int)-1].shape

In [ ]:
np.array(y1, dtype=int)-1

In [ ]:
y1

In [ ]:
lossfn(y1[i], model.predict_on_batch(X[i]).numpy().reshape(-1))

In [ ]:
lossfn(y1[i], model.predict_on_batch(X[i]))

In [ ]:
inss = keras.layers.Input(shape=(182,))
xs = keras.layers.Dense(150)(inss)
xs = keras.activations.tanh(xs)

xs = keras.layers.Dense(150)(xs)
xs = keras.layers.ReLU()(xs)

xs = keras.layers.Dense(100)(xs)
xs = keras.layers.ReLU()(xs)

xs_ = keras.layers.Dense(75)(xs)
xs = keras.layers.ReLU()(xs_)

xs = keras.layers.Dense(40)(xs)
xs = keras.activations.relu(xs)

xs = keras.layers.Dense(1)(xs)
xs = keras.layers.ReLU()(xs)
#p = keras.layers.Dense(3)(x_)
#p = keras.activations.softmax(p)

smallmodel = keras.Model(inputs=inss, outputs=xs)
smallmodel.compile(loss=keras.losses.poisson, optimizer=keras.optimizers.Adam()) # keras.losses.mean_squared_error

In [ ]:
# Set up an endless iteration through epochs and batches of data
BATCH_SIZE = 100
indices = np.arange(len(T))
indices = np.array_split(indices, len(T) / BATCH_SIZE)
eb_indices = enumerate_cycle(indices)

# Accumulate some stats as we go along
loss_history_s = []
dev_loss_history_small = []

In [ ]:
while not interrupted():
    if(len(loss_history_s) > 0 and loss_history_s[-1][0]-loss_history_s[0][0] >= 45):
        break
    (epoch,batch),i = next(eb_indices)
    #print(model.predict_on_batch(X[i]))
    #input()
    loss = smallmodel.train_on_batch(T[i], ytrain[i])
    if batch % 100 == 0:
        IPython.display.clear_output(wait=True)
        print(f'loss={loss} epoch={epoch} batchnum={batch}/{len(indices)}' + f' time={loss_history_s[-1][0]-loss_history_s[0][0]}' if epoch > 0 else '')
        loss_history_s.append((time.time(), epoch+batch/len(indices), loss))
    if batch % 200 == 0:
        devloss = tf.reduce_mean(keras.losses.poisson(ydev.reshape(-1,1), smallmodel.predict_on_batch(D)))
        dev_loss_history_small.append((time.time(), epoch+batch/len(indices), devloss))

In [ ]:
# Simple plot of the training loss
df = pandas.DataFrame.from_records(loss_history_s, columns=['time','epoch','loss'])
df2 = pandas.DataFrame.from_records(dev_loss_history_small, columns=['time','epoch','loss'])
df.time = df.time - df.time[0]
df2.time = df2.time - df2.time[0]
print("Final training loss:", df.loss.ewm(com=5).mean().iloc[-1])
fig,ax = plt.subplots(figsize=(6,2))
ax.plot(df.time, df.loss, alpha=0.1)
ax.plot(df.time, df.loss.ewm(com=5).mean(), linewidth=1)
ax.plot(df2.time, df2.loss)
#ax.set_ylim([-3.5,0])
plt.show()
df.time[-5:]

In [ ]:
val_rec_cat_prob[293]

In [ ]:
yps_ = pca_cat_50.predict_on_batch(V_50).numpy()
#ys_ = (yps_.argmax(axis=1) + 1)

fig, ax = plt.subplots()
for _y in np.arange(1,15):
    inds = yval == _y
    ys_rv = yps_[inds]
    print(ys_rv.shape)
    responsibilities = ys_rv.mean(axis=0)
    print(responsibilities)
    for i in np.arange(1,15):
        ax.fill([_y-0.4875, _y-0.4875, _y+0.4875, _y+0.4875], [i-0.4875, i+0.4875, i+0.4875, i-0.4875], c='black', alpha=responsibilities[i-1])
        

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_color('white')
ax.spines['bottom'].set_color('white')
list(map(lambda ts: ts.set_color('white'),ax.xaxis.get_majorticklines()))
list(map(lambda ts: ts.set_color('white'),ax.yaxis.get_majorticklines()))
plt.show()

In [ ]:
yps_ = med_model_cat.predict_on_batch(V).numpy()
ys_ = (yps_.argmax(axis=1) + 1)

fig, ax = plt.subplots()
for _y in np.arange(1,15):
    inds = yval == _y
    ys_rv = yps_[inds]
   # print(ys_rv.shape)
    responsibilities = ys_rv.mean(axis=0)
   # print(responsibilities)
    for i in np.arange(1,15):
        ax.fill([_y-0.5, _y-0.5, _y+0.5, _y+0.5], [i-0.5, i+0.5, i+0.5, i-0.5], c='black', alpha=responsibilities[i-1])
        
        

In [ ]:
from tqdm import tqdm_notebook as tqdm
, 
yps_ = val_rec_poiss
ys_ = med_model_poisson.predict_on_batch(D).numpy()

fig, ax = plt.subplots()

for (y, y_) in tqdm(zip (yval_c, yps_)):
    if interrupted(): break
    ax.plot([y-0.5, y+0.5], [y_, y_], linewidth=0.5, c='black', alpha=200/(ydev==y).sum())

ax.set_ylim([-0.5, 14.5])
ax.set_xlim([-0.5, 14.5])
    
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_color('white')
ax.spines['bottom'].set_color('white')
list(map(lambda ts: ts.set_color('white'),ax.xaxis.get_majorticklines()))
list(map(lambda ts: ts.set_color('white'),ax.yaxis.get_majorticklines()))
plt.show()
"""
fig, ax = plt.subplots()
for _y in np.arange(1,15):
    inds = ytrain == _y
    ys_rv = yps_[inds]
   # print(ys_rv.shape)
    responsibilities = ys_rv.mean(axis=0)
   # print(responsibilities)
    for i in np.arange(1,15):
        ax.fill([_y-0.5, _y-0.5, _y+0.5, _y+0.5], [i-0.5, i+0.5, i+0.5, i-0.5], c='black', alpha=responsibilities[i-1])
    """

In [ ]:
insm = keras.layers.Input(shape=(121,))
xm = keras.layers.Dense(175)(insm)
xm = keras.activations.relu(xm)

xm = keras.layers.Dense(225)(xm)
xm = keras.activations.exponential(xm)

xm = keras.layers.Dense(225)(xm)
xm = keras.activations.relu(xm)

xm = keras.layers.Dense(150)(xm)
xm = keras.activations.relu(xm)

xm = keras.layers.Dense(110)(xm)
xm = keras.activations.relu(xm)

xm_ = keras.layers.Dense(80)(xm)
xm = keras.activations.relu(xm_)

xm = keras.layers.Dense(50)(xm)
xm = keras.activations.relu(xm)

xm = keras.layers.Dense(1)(xm)
xm = keras.layers.Lambda(lambda x: keras.activations.elu(x)+1)(xm)
#p = keras.layers.Dense(3)(x_)
#p = keras.activations.softmax(p)

medmodel = keras.Model(inputs=insm, outputs=xm)
medmodel.compile(loss=lossfn, optimizer=keras.optimizers.Adam()) # keras.losses.mean_squared_error

In [ ]:
# Set up an endless iteration through epochs and batches of data
BATCH_SIZE = 100
indices = np.arange(len(T))
indices = np.array_split(indices, len(T) / BATCH_SIZE)
eb_indices = enumerate_cycle(indices)

# Accumulate some stats as we go along
loss_history_m = []
dev_loss_history_med = []

In [ ]:
while not interrupted():
    if(len(loss_history_m) > 0 and loss_history_m[-1][0]-loss_history_m[0][0] >= 45):
        break
    (epoch,batch),i = next(eb_indices)
    #print(model.predict_on_batch(X[i]))
    #input()
    loss = medmodel.train_on_batch(T[i], ytrain[i])
    if batch % 100 == 0:
        IPython.display.clear_output(wait=True)
        print(f'loss={loss} epoch={epoch} batchnum={batch}/{len(indices)}' + f' time={loss_history_m[-1][0]-loss_history_m[0][0]}' if epoch > 0 else '')
        loss_history_m.append((time.time(), epoch+batch/len(indices), loss))
    if batch % 200 == 0:
        devloss = tf.reduce_mean(lossfn(ydev.reshape(-1,1), medmodel.predict_on_batch(D)))
        dev_loss_history_med.append((time.time(), epoch+batch/len(indices), devloss))

In [ ]:
# Simple plot of the training loss
dfs = pandas.DataFrame.from_records(loss_history_l, columns=['time','epoch','loss'])
df2s = pandas.DataFrame.from_records(dev_loss_history_l, columns=['time','epoch','loss'])
#df = pandas.DataFrame.from_records(loss_history_s, columns=['time','epoch','loss'])
#df2 = pandas.DataFrame.from_records(dev_loss_history_small, columns=['time','epoch','loss'])
#df.time = df.time - df.time[0]
#df2.time = df2.time - df2.time[0]
dfs.time = dfs.time - dfs.time[0]
df2s.time = df2s.time - df2s.time[0]
#print("Final training loss:", df.loss.ewm(com=5).mean().iloc[-1])
fig,ax = plt.subplots(figsize=(6,2))
#ax.plot(df.time, df.loss, alpha=0.1)
#ax.plot(df.time, df.loss.ewm(com=5).mean(), linewidth=1)
#ax.plot(df2.time, df2.loss.ewm(com=5).mean())

ax.plot(dfs.time, dfs.loss, alpha=0.1)
ax.plot(dfs.time, dfs.loss.ewm(com=5).mean(), linewidth=1)
ax.plot(df2s.time, df2s.loss.ewm(com=5).mean())
ax.set_ylim([-1,3])
plt.show()
#df.time[-5:]

In [ ]:
np.mean(keras.losses.poisson(ytrain[i].reshape(-1,1), lmodel.predict_on_batch(T[i])))

In [ ]:
i = (keras.layers.Input(1))
a = keras.layers.Lambda(lambda x: keras.activations.elu(x)+1)(i)
m = keras.models.Model(inputs=i, outputs=a)
m.compile(loss=lambda x,y : 0.0, optimizer=keras.optimizers.Adam())
m.predict_on_batch([-11])

In [ ]:
insl = keras.layers.Input(shape=(182,))
xl = keras.layers.Dense(500)(insl)
xl = keras.activations.relu(xl)

xl = keras.layers.Dense(400)(xl)
xl = keras.activations.tanh(xl)

xl = keras.layers.Dense(300)(xl)
xl = keras.activations.relu(xl)

xl = keras.layers.Dense(275)(xl)
xl = keras.activations.relu(xl)

xl = keras.layers.Dense(225)(xl)
xl = keras.activations.relu(xl)

xl = keras.layers.Dense(150)(xl)
xl = keras.activations.relu(xl)

xl = keras.layers.Dense(110)(xl)
xl = keras.activations.relu(xl)

xl_ = keras.layers.Dense(80)(xl)
xl = keras.activations.relu(xl_)

xl = keras.layers.Dense(10)(xl)
xl = keras.activations.relu(xl)

xl = keras.layers.Dense(1)(xl)
xl = keras.layers.Lambda(lambda x: keras.activations.elu(x-1, alpha=0.1)+1)(xl)

lmodel = keras.Model(inputs=insl, outputs=xl)
lmodel.compile(loss=keras.losses.poisson, optimizer=keras.optimizers.Adam()) # keras.losses.mean_squared_error

In [ ]:
BATCH_SIZE = 100
indices = np.arange(len(T))
indices = np.array_split(indices, len(T) / BATCH_SIZE)
eb_indices = enumerate_cycle(indices)

# Accumulate some stats as we go along
loss_history_l = []
dev_loss_history_l = []
dev_loss_history_l_fs = []

In [ ]:
while not interrupted():
    if(len(loss_history_l) > 0 and loss_history_l[-1][0]-loss_history_l[0][0] >= 45):
        break
    (epoch,batch),i = next(eb_indices)
    #print(model.predict_on_batch(X[i]))
    #input()
    loss = lmodel.train_on_batch(T[i], ytrain[i])
    if np.isnan(loss): break
    if batch % 100 == 0:
        IPython.display.clear_output(wait=True)
        print(f'loss={loss} epoch={epoch} batchnum={batch}/{len(indices)}' + f' time={loss_history_l[-1][0]-loss_history_l[0][0]}' if epoch > 0 else '')
        loss_history_l.append((time.time(), epoch+batch/len(indices), loss))
    if batch % 200 == 0:
        devloss = lmodel.train_on_batch(D)
        dev_loss_history_l.append((time.time(), epoch+batch/len(indices), devloss))

In [ ]:
lmodel.predict_on_batch(V)

In [ ]:
np.mean(T)+7*np.std(T)

In [ ]:
T, D, V = prepared_data2[train], prepared_data2[dev], prepared_data2[val] 
ytrain, ydev, yval = y1[train], y1[dev], y1[val]

In [ ]:
list(map(lambda l: l[-1], loss_history_l))

In [ ]:
insl = keras.layers.Input(shape=(182,))
xl = keras.layers.Dense(500)(insl)
xl = keras.activations.relu(xl)

xl = keras.layers.Dense(400)(xl)
xl = keras.activations.tanh(xl)

xl = keras.layers.Dense(300)(xl)
xl = keras.activations.relu(xl)

xl = keras.layers.Dense(275)(xl)
xl = keras.activations.relu(xl)

xl = keras.layers.Dense(225)(xl)
xl = keras.activations.relu(xl)

xl = keras.layers.Dense(150)(xl)
xl = keras.activations.relu(xl)

xl = keras.layers.Dense(110)(xl)
xl = keras.activations.relu(xl)

xl_ = keras.layers.Dense(80)(xl)
xl = keras.activations.relu(xl_)

xl = keras.layers.Dense(40)(xl)
xl = keras.activations.relu(xl)

xl = keras.layers.Dense(14)(xl)
xl = keras.activations.softmax(xl)

lmodel = keras.Model(inputs=insl, outputs=xl)
lmodel.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam()) # keras.losses.mean_squared_error

In [ ]:
yprob = np.repeat(np.arange(14).reshape(1,-1), len(y1), axis=0)
yprob = yprob == np.repeat(y1-1, 14).reshape(-1,14)

ytprob = np.repeat(np.arange(14).reshape(1,-1), len(ytrain), axis=0)
ytprob[:, :] = ytprob == np.repeat(ytrain-1, 14).reshape(-1,14)
ydprob = np.repeat(np.arange(14).reshape(1,-1), len(ydev), axis=0)
ydprob[:, :] = ydprob == np.repeat(ydev-1, 14).reshape(-1,14)
yvprob = np.repeat(np.arange(14).reshape(1,-1), len(yval), axis=0)
yvprob[:, :] = yvprob == np.repeat(yval-1, 14).reshape(-1,14)

In [ ]:
while not interrupted():
    if(len(loss_history_l) > 0 and loss_history_l[-1][0]-loss_history_l[0][0] >= 1800):
        break
    (epoch,batch),i = next(eb_indices)
    #print(model.predict_on_batch(X[i]))
    #input()
    loss = lmodel.train_on_batch(T[i], ytprob[i])
    if np.isnan(loss): break
    if batch % 100 == 0:
        IPython.display.clear_output(wait=True)
        print(f'loss={loss} epoch={epoch} batchnum={batch}/{len(indices)}' + f' time={loss_history_l[-1][0]-loss_history_l[0][0]}' if epoch > 0 else '')
        loss_history_l.append((time.time(), epoch+batch/len(indices), loss))
    if batch % 200 == 0:
        sample = np.random.choice(np.arange(len(D)), 10*BATCH_SIZE, replace=False)
        devloss = tf.reduce_mean(keras.losses.categorical_crossentropy(ydprob[sample], lmodel.predict_on_batch(D[sample])))
        dev_loss_history_l.append((time.time(), epoch+batch/len(indices), devloss))
        if len(dev_loss_history_l) > 20 and dev_loss_history_l[-1][-1] > dev_loss_history_l[-2][-1] and dev_loss_history_l[-2][-1] > dev_loss_history_l[-3][-1] and ((dev_loss_history_l[-1][-1] > dev_loss_history_l[-3][-1] + 0.075) or (dev_loss_history_l[-1][-1] > min(map(lambda x: x[-1], dev_loss_history_l)) + 0.1)):
            print("FAAIIIIL", list(map(lambda x: x[-1], dev_loss_history_l)))
            dev_loss_history_l_fs.append((time.time(), epoch+batch/len(indices), devloss))

In [ ]:
dev_loss_history_l

In [ ]:
i

In [ ]:
lmodel.train_on_batch(T[i], ytprob[i])

In [ ]:
2.3079793 -  2.2988513

In [ ]:
np.array([[1,4,3], [4,8,6], [7,12,9]]).mean(axis=0)

In [ ]:
fig, ax = plt.subplots()

cs = np.random.random(14)
cs /= cs.sum()

for i in np.arange(1,15):
    ax.fill([0,1,1,0], [i-1, i, i, i-1], c='black', alpha=cs[i-1])


# ACTUAL MEASUREMENTS...

## MLP

Divide as follows:

 - poisson loss
   - small network
   - med network
   - big network
 - cat loss
   - small ...

In [ ]:
# preliminaries
ins = keras.layers.Input(182)
poisson = keras.losses.poisson
cat = keras.losses.categorical_crossentropy
Adam = keras.optimizers.Adam

In [ ]:
# small model
def regenerate_small():
    x_small = keras.layers.Dense(150)(ins)
    x_small = keras.layers.ReLU()(x_small)

    x_small = keras.layers.Dense(150)(x_small)
    x_small = keras.layers.ReLU()(x_small)

    x_small = keras.layers.Dense(100)(x_small)
    x_small = keras.layers.ReLU()(x_small)

    x_small = keras.layers.Dense(75)(x_small)
    x_small = keras.layers.ReLU()(x_small)

    x_small = keras.layers.Dense(40)(x_small)
    x_small = keras.activations.relu(x_small)

    poisson_out_small = keras.layers.Dense(1, kernel_regularizer=keras.regularizers.l2())(x_small)
    poisson_out_small = keras.layers.ReLU()(poisson_out_small)

    cat_out_small = keras.layers.Dense(14)(x_small)
    cat_out_small = keras.activations.softmax(cat_out_small)
    
    small_model_poisson = keras.models.Model(inputs=ins, outputs=poisson_out_small)
    small_model_poisson.compile(loss=poisson, optimizer=Adam())

    small_model_cat = keras.models.Model(inputs=ins, outputs=cat_out_small)
    small_model_cat.compile(loss=cat, optimizer=Adam())
    
    return small_model_poisson, small_model_cat

In [ ]:
small_model_poisson = keras.models.Model(inputs=ins, outputs=poisson_out_small)
small_model_poisson.compile(loss=poisson, optimizer=Adam())

small_model_cat = keras.models.Model(inputs=ins, outputs=cat_out_small)
small_model_cat.compile(loss=cat, optimizer=Adam())

In [ ]:
# Set up an endless iteration through epochs and batches of data

def experiment(model, lossmthd, trainy, devy, maxtime):
    BATCH_SIZE = 100
    indices = np.arange(len(T))
    indices = np.array_split(indices, len(T) / BATCH_SIZE)
    eb_indices = enumerate_cycle(indices)

    # Accumulate some stats as we go along
    loss_history_l = []
    dev_loss_history_l = []
    dev_loss_history_l_fs = []

    model.compile(loss=lossmthd, optimizer=Adam())
    
    while not interrupted():

        
        if(len(loss_history_l) > 0 and loss_history_l[-1][0]-loss_history_l[0][0] >= maxtime):
            break
        (epoch,batch),i = next(eb_indices)
        #print(model.predict_on_batch(X[i]))
        #input()
        loss = model.train_on_batch(T[i], trainy[i])
        if np.isnan(loss): break
        if batch % 100 == 0:
            IPython.display.clear_output(wait=True)
            print(f'loss={loss} epoch={epoch} batchnum={batch}/{len(indices)}' + f' time={loss_history_l[-1][0]-loss_history_l[0][0]}' if epoch > 0 else '')
            loss_history_l.append((time.time(), epoch+batch/len(indices), loss))
            sample = np.random.choice(np.arange(len(D)), 4*BATCH_SIZE, replace=False)
            print(ydprob[sample].shape,  model.predict_on_batch(D[sample]).numpy().shape)
            devloss = tf.reduce_mean(lossmthd(devy[sample], model.predict_on_batch(D[sample])))
            dev_loss_history_l.append((time.time(), epoch+batch/len(indices), devloss))
            if len(dev_loss_history_l) > 20 and dev_loss_history_l[-1][-1] > dev_loss_history_l[-2][-1] and dev_loss_history_l[-2][-1] > dev_loss_history_l[-3][-1] and ((dev_loss_history_l[-1][-1] > dev_loss_history_l[-3][-1] + 0.075) or (dev_loss_history_l[-1][-1] > min(map(lambda x: x[-1], dev_loss_history_l)) + 0.1)):
                print("FAAIIIIL")
                dev_loss_history_l_fs.append((time.time(), epoch+batch/len(indices), devloss))
            else:
                print()
                                  
    return loss_history_l, dev_loss_history_l, dev_loss_history_l_fs

In [ ]:
small_model_poisson, small_model_cat = regenerate_small()
losses_small_poisson, dev_losses_small_poisson, fails_small_poisson = experiment(small_model_poisson, poisson, ytrain, ydev, 120)
small_model_poisson, small_model_cat = regenerate_small()
losses_small_cat, dev_losses_small_cat, fails_small_cat = experiment(small_model_cat, cat, ytprob, ydprob, 120)

In [ ]:
def draw_losses(losses, dev_losses, fails, filename, yaxis=False, comp=10):
    devdf = pandas.DataFrame.from_records(losses, columns=['time','epoch','loss'])
    df2s = pandas.DataFrame.from_records(dev_losses, columns=['time','epoch','loss'])
    devdf.time = devdf.time - devdf.time[0]
    df2s.time = df2s.time - df2s.time[0]
    fig,ax = plt.subplots(figsize=(8,4))
    
    t1 = devdf[devdf.epoch == 1].time.values[0]
    t2 = devdf[devdf.epoch == 1].time.values[-1]
    
    ax.axvspan(t1, t2, color='gray', alpha=0.07)

    ax.plot(devdf.time, devdf.loss.values, alpha=0.1, c='black')
    ax.plot(devdf.time, devdf.loss.ewm(com=comp).mean(), linewidth=1, c='black', label='Training Loss')
    ax.plot(df2s.time, df2s.loss.ewm(com=comp).mean(), c='#66c2a5', label='Development Loss')
    
   # for i,f in enumerate(fails):
   #     t, b, l = f
   #     ax.plot([t-losses[0][0], t-losses[0][0]], [-100, 100], c='#fc8d62', linewidth=0.5)
    upper_lim = devdf.loss.append(df2s.loss).values.reshape(-1).argsort()[int(0.975*(len(devdf) + len(df2s)))]
    upper_lim = float(devdf.loss.append(df2s.loss).values[upper_lim])
    lower_lim = devdf.loss.append(df2s.loss).values.reshape(-1).argsort()[int(0.01*(len(devdf) + len(df2s)))]
    lower_lim = float(devdf.loss.append(df2s.loss).values[lower_lim])
    print(upper_lim)
    ax.set_ylim([lower_lim, upper_lim + 0.1*abs(upper_lim)])
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    if yaxis:
        ax.spines['left'].set_color("white")
        ax.spines['bottom'].set_color("white")
    else:
        ax.spines['left'].set_visible(False)
        ax.spines['bottom'].set_visible(False)
    list(map(lambda ts: ts.set_color('white'),ax.xaxis.get_majorticklines()))
    list(map(lambda ts: ts.set_color('white'),ax.yaxis.get_majorticklines()))
    ax.set_xlabel('training time in seconds')
    if yaxis:
        ax.set_ylabel(r'$\ln\mathrm{lik}(\theta | \mathbf{y}) - c$', rotation=0, horizontalalignment='right')
    fig.tight_layout()
    plt.legend(frameon=False)
    plt.savefig(filename)
    plt.show()
    print(devdf.time.values[-1])

In [ ]:
draw_losses(losses_small_poisson, dev_losses_small_poisson, fails_small_poisson, 'figures/basic_small_poisson.pdf', yaxis=True)

In [ ]:
_, small_model_cat = regenerate_small()
losses_small_cat, dev_losses_small_cat, _ = experiment(small_model_cat, cat, ytprob, ydprob, 120)
draw_losses(losses_small_cat, dev_losses_small_cat, fails_small_poisson, 'figures/basic_small_cat.pdf', yaxis=False)

In [ ]:
draw_losses(losses_small_cat, dev_losses_small_cat, fails_small_poisson, 'figures/basic_small_cat.pdf', yaxis=True)

In [ ]:
def regenerate_medium():
    x_med = keras.layers.Dense(200)(ins)
    x_med = keras.layers.ReLU()(x_med)

    
    x_med = keras.layers.Dense(225)(x_med)
    x_med = keras.layers.ReLU()(x_med)

    x_med = keras.layers.Dense(185)(x_med)
    x_med = keras.layers.ReLU()(x_med)
    
    x_med = keras.layers.Dense(150)(x_med)
    x_med = keras.layers.ReLU()(x_med)

    x_med = keras.layers.Dense(110)(x_med)
    x_med = keras.layers.ReLU()(x_med)

    x_med = keras.layers.Dense(85)(x_med)
    x_med = keras.layers.ReLU()(x_med)

    x_med = keras.layers.Dense(50)(x_med)
    x_med = keras.activations.relu(x_med)

    poisson_out_med = keras.layers.Dense(1)(x_med)
    poisson_out_med = keras.layers.ReLU()(poisson_out_med)

    cat_out_med = keras.layers.Dense(14)(x_med)
    cat_out_med = keras.activations.softmax(cat_out_med)
    
    med_model_poisson = keras.models.Model(inputs=ins, outputs=poisson_out_med)
    med_model_poisson.compile(loss=poisson, optimizer=Adam())

    med_model_cat = keras.models.Model(inputs=ins, outputs=cat_out_med)
    med_model_cat.compile(loss=cat, optimizer=Adam())
    
    return med_model_poisson, med_model_cat

In [ ]:
med_model_poisson, _ = regenerate_medium()

losses_med_poisson, dev_losses_med_poisson, f = experiment(med_model_poisson, poisson,ytrain, ydev, 120)

med_model_poisson, med_model_cat = regenerate_medium()
losses_med_cat, dev_losses_med_cat, f = experiment(med_model_cat, cat, ytprob, ydprob, 120)

In [ ]:
draw_losses(losses_med_poisson, dev_losses_med_poisson, f, 'figures/basic_med_poisson.pdf')
draw_losses(losses_med_cat, dev_losses_med_cat, f, 'figures/basic_med_cat.pdf')

In [ ]:
def regenerate_large():
    x_med = keras.layers.Dense(375)(ins)
    x_med = keras.layers.ReLU()(x_med)

    
    x_med = keras.layers.Dense(450)(x_med)
    x_med = keras.layers.ReLU()(x_med)

    x_med = keras.layers.Dense(450)(x_med)
    x_med = keras.layers.ReLU()(x_med)
    
    x_med = keras.layers.Dense(325)(x_med)
    x_med = keras.layers.ReLU()(x_med)

    x_med = keras.layers.Dense(275)(x_med)
    x_med = keras.layers.ReLU()(x_med)
    
    x_med = keras.layers.Dense(130)(x_med)
    x_med = keras.layers.ReLU()(x_med)

    x_med = keras.layers.Dense(115)(x_med)
    x_med = keras.layers.ReLU()(x_med)
    
    x_med = keras.layers.Dense(90)(x_med)
    x_med = keras.layers.ReLU()(x_med)

    x_med = keras.layers.Dense(75)(x_med)
    x_med = keras.layers.ReLU()(x_med)

    x_med = keras.layers.Dense(50)(x_med)
    x_med = keras.activations.relu(x_med)

    poisson_out_med = keras.layers.Dense(1)(x_med)
    poisson_out_med = keras.layers.ReLU()(poisson_out_med)

    cat_out_med = keras.layers.Dense(14)(x_med)
    cat_out_med = keras.activations.softmax(cat_out_med)
    
    med_model_poisson = keras.models.Model(inputs=ins, outputs=poisson_out_med)
    med_model_poisson.compile(loss=poisson, optimizer=Adam())

    med_model_cat = keras.models.Model(inputs=ins, outputs=cat_out_med)
    med_model_cat.compile(loss=cat, optimizer=Adam())
    
    return med_model_poisson, med_model_cat

In [ ]:
large_model_poisson, _ = regenerate_large()

losses_large_poisson, dev_losses_large_poisson, f = experiment(large_model_poisson, poisson, ytrain, ydev, 120)
                                                              
_, large_model_cat = regenerate_large()                                                              
losses_large_cat, dev_losses_large_cat, f = experiment(large_model_cat, cat, ytprob, ydprob, 120)

In [ ]:
draw_losses(losses_large_poisson, dev_losses_large_poisson, f, 'figures/basic_large_poisson.pdf')
draw_losses(losses_large_cat, dev_losses_large_cat, f, 'figures/basic_large_cat.pdf')

In [ ]:
draw_comparison(losses_small_poisson, losses_med_poisson, losses_large_poisson, 'figures/comparison_poisson_mlp.pdf', yaxis=True)

In [ ]:
draw_comparison(losses_small_cat, losses_med_cat, losses_large_cat, 'figures/comparison_cat_mlp.pdf')

In [ ]:
def regenerate_pca(dim):
    ins = keras.layers.Input(dim)
    
    x_med = keras.layers.Dense(375)(ins)
    x_med = keras.layers.ReLU()(x_med)

    
    x_med = keras.layers.Dense(450, kernel_regularizer=keras.regularizers.l2(0.05))(x_med)
    x_med = keras.layers.ReLU()(x_med)

    x_med = keras.layers.Dense(450)(x_med)
    x_med = keras.layers.ReLU()(x_med)
    
    x_med = keras.layers.Dense(325, kernel_regularizer=keras.regularizers.l2(0.1))(x_med)
    x_med = keras.layers.ReLU()(x_med)

    x_med = keras.layers.Dense(275)(x_med)
    x_med = keras.layers.ReLU()(x_med)
    
    x_med = keras.layers.Dense(130)(x_med)
    x_med = keras.layers.ReLU()(x_med)

    x_med = keras.layers.Dense(115, kernel_regularizer=keras.regularizers.l2(0.05))(x_med)
    x_med = keras.layers.ReLU()(x_med)
    
    x_med = keras.layers.Dense(90)(x_med)
    x_med = keras.layers.ReLU()(x_med)

    x_med = keras.layers.Dense(75)(x_med)
    x_med = keras.layers.ReLU()(x_med)

    x_med = keras.layers.Dense(50)(x_med)
    x_med = keras.activations.relu(x_med)

    poisson_out_med = keras.layers.Dense(1)(x_med)
    poisson_out_med = keras.layers.ReLU()(poisson_out_med)

    cat_out_med = keras.layers.Dense(14)(x_med)
    cat_out_med = keras.activations.softmax(cat_out_med)
    
    med_model_poisson = keras.models.Model(inputs=ins, outputs=poisson_out_med)
    med_model_poisson.compile(loss=poisson, optimizer=Adam())

    med_model_cat = keras.models.Model(inputs=ins, outputs=cat_out_med)
    med_model_cat.compile(loss=cat, optimizer=Adam())
    
    return med_model_poisson, med_model_cat

In [ ]:
from sklearn.decomposition import PCA

pca = PCA()

pca_res = pca.fit_transform(T)

T_50, T_85, T_125 = pca_res[:, :50], pca_res[:, :85], pca_res[:, :125]

DPCA, VPCA = pca.transform(D), pca.transform(V)

D_50, D_85, D_125 = DPCA[:, :50], DPCA[:, :85], DPCA[:, :125]
V_50, V_85, V_125 = VPCA[:, :50], VPCA[:, :85], VPCA[:, :125]

In [ ]:
pca_poiss_50, pca_cat_50 = regenerate_pca(50)
pca_poiss_85 , pca_cat_85 = regenerate_pca(85)
pca_poiss_125, pca_cat_125 = regenerate_pca(125)

In [ ]:
def pca_experiment(model, lossmthd, trainx, trainy, devx, devy, maxtime):
    BATCH_SIZE = 100
    indices = np.arange(len(trainx))
    indices = np.array_split(indices, len(trainx) / BATCH_SIZE)
    eb_indices = enumerate_cycle(indices)

    # Accumulate some stats as we go along
    loss_history_l = []
    dev_loss_history_l = []
    dev_loss_history_l_fs = []

    model.compile(loss=lossmthd, optimizer=Adam())
    while not interrupted():
        if(len(loss_history_l) > 0 and loss_history_l[-1][0]-loss_history_l[0][0] >= maxtime):
            break
        (epoch,batch),i = next(eb_indices)
        #print(model.predict_on_batch(X[i]))
        #input()
        loss = model.train_on_batch(trainx[i], trainy[i])
        if np.isnan(loss): break
        if batch % 100 == 0:
            IPython.display.clear_output(wait=True)
            print(f'loss={loss} epoch={epoch} batchnum={batch}/{len(indices)}' + f' time={loss_history_l[-1][0]-loss_history_l[0][0]}' if epoch > 0 else '')
            loss_history_l.append((time.time(), epoch+batch/len(indices), loss))
            sample = np.random.choice(np.arange(len(devx)), 4*BATCH_SIZE, replace=False)
            print(ydprob[sample].shape,  model.predict_on_batch(devx[sample]).numpy().shape)
            devloss = tf.reduce_mean(lossmthd(devy[sample], model.predict_on_batch(devx[sample])))
            dev_loss_history_l.append((time.time(), epoch+batch/len(indices), devloss))
            if len(dev_loss_history_l) > 20 and dev_loss_history_l[-1][-1] > dev_loss_history_l[-2][-1] and dev_loss_history_l[-2][-1] > dev_loss_history_l[-3][-1] and ((dev_loss_history_l[-1][-1] > dev_loss_history_l[-3][-1] + 0.075) or (dev_loss_history_l[-1][-1] > min(map(lambda x: x[-1], dev_loss_history_l)) + 0.1)):
                print("FAAIIIIL")
                dev_loss_history_l_fs.append((time.time(), epoch+batch/len(indices), devloss))
            else:
                print()
                                  
    return loss_history_l, dev_loss_history_l

In [ ]:
pca_poiss_50, pca_cat_50 = regenerate_pca( 50)    ; pca_50_poiss_loss, pca_50_poiss_devloss = pca_experiment(pca_poiss_50, poisson, T_50, ytrain, D_50, ydev, 120)
pca_poiss_85, pca_cat_85 = regenerate_pca( 85)    ; pca_85_poiss_loss, pca_85_poiss_devloss = pca_experiment(pca_poiss_85, poisson, T_85, ytrain, D_85, ydev, 120)
pca_poiss_125, pca_cat_125 = regenerate_pca(125)  ; pca_125_poiss_loss, pca_125_poiss_devloss = pca_experiment(pca_poiss_125, poisson, T_125, ytrain, D_125, ydev, 120)

In [ ]:
pca_poiss_50, pca_cat_50 = regenerate_pca( 50)    ;pca_50_cat_loss, pca_50_cat_devloss = pca_experiment(pca_cat_50, cat, T_50, ytprob, D_50, ydprob, 120)
pca_poiss_85, pca_cat_85 = regenerate_pca( 85)    ;pca_85_cat_loss, pca_85_cat_devloss = pca_experiment(pca_cat_85, cat, T_85, ytprob, D_85, ydprob, 120)
pca_poiss_125, pca_cat_125 = regenerate_pca(125)  ;pca_125_cat_loss, pca_125_cat_devloss = pca_experiment(pca_cat_125, cat, T_125, ytprob, D_125, ydprob, 120)


In [ ]:
def draw_pca_losses(losses, dev_losses, fails, filename, yaxis=False):
    devdf = pandas.DataFrame.from_records(losses, columns=['time','epoch','loss'])
    df2s = pandas.DataFrame.from_records(dev_losses, columns=['time','epoch','loss'])
    devdf.time = devdf.time - devdf.time[0]
    df2s.time = df2s.time - df2s.time[0]
    fig,ax = plt.subplots(figsize=(8,4))

    ax.plot(devdf.time, devdf.loss, alpha=0.1, c='black')
    ax.plot(devdf.time, devdf.loss.ewm(com=10).mean(), linewidth=1, c='black')
    ax.plot(df2s.time, df2s.loss.ewm(com=10).mean(), c='#66c2a5')
    
   # for i,f in enumerate(fails):
   #     t, b, l = f
   #     ax.plot([t-losses[0][0], t-losses[0][0]], [-100, 100], c='#fc8d62', linewidth=0.5)
    upper_lim = devdf.loss.append(df2s.loss).values.reshape(-1).argsort()[int(0.975*(len(devdf) + len(df2s)))]
    upper_lim = float(devdf.loss.append(df2s.loss).values[upper_lim])
    print(upper_lim)
    ax.set_ylim([devdf.loss.values.min() - 0.1*abs(devdf.loss.values.min()), upper_lim + 0.1*abs(upper_lim)])
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    if yaxis:
        ax.spines['left'].set_color("white")
        ax.spines['bottom'].set_color("white")
    else:
        ax.spines['left'].set_visible(False)
        ax.spines['bottom'].set_visible(False)
    list(map(lambda ts: ts.set_color('white'),ax.xaxis.get_majorticklines()))
    list(map(lambda ts: ts.set_color('white'),ax.yaxis.get_majorticklines()))
    ax.set_xlabel('training time in seconds')
    if yaxis:
        ax.set_ylabel(r'$\ln\mathrm{lik}(\theta | \mathbf{y}) - c$', rotation=0, horizontalalignment='right')
    fig.tight_layout()
    plt.savefig(filename)
    plt.show()
    print(devdf.time.values[-1])

In [ ]:
draw_pca_losses(pca_50_poiss_loss, pca_50_poiss_devloss, [], 'figures/pca_50_poiss.pdf', yaxis=True)
draw_pca_losses(pca_85_poiss_loss, pca_85_poiss_devloss, [], 'figures/pca_85_poiss.pdf')
draw_pca_losses(pca_125_poiss_loss, pca_125_poiss_devloss, [], 'figures/pca_125_poiss.pdf')


In [ ]:
draw_pca_losses(pca_50_cat_loss, pca_50_cat_devloss, [], 'figures/pca_50_cat.pdf', yaxis=True)
draw_pca_losses(pca_85_cat_loss, pca_85_cat_devloss, [], 'figures/pca_85_cat.pdf')
draw_pca_losses(pca_125_cat_loss, pca_125_cat_devloss, [], 'figures/pca_125_cat.pdf')


In [ ]:
def draw_pca_comparison(small, med, large, old, filename, yaxis=False, legend=True):
    dfsmall = pandas.DataFrame.from_records(small, columns=['time','epoch','loss'])
    dfmed = pandas.DataFrame.from_records(med, columns=['time','epoch','loss'])
    dflrg = pandas.DataFrame.from_records(large, columns=['time','epoch','loss'])
    dfold = pandas.DataFrame.from_records(old, columns=['time','epoch','loss'])
    dfsmall.time = dfsmall.time - dfsmall.time[0]
    dfmed.time = dfmed.time - dfmed.time[0]
    dflrg.time = dflrg.time - dflrg.time[0]
    dfold.time = dfold.time - dfold.time[0] if len(dfold) > 0 else dfold.time
    fig,ax = plt.subplots(figsize=(8,4))

    ax.plot(dfsmall.time, dfsmall.loss.ewm(com=10).mean(), linewidth=0.75, c='#a6cee3', label='50 Dimensions')
    ax.plot(dfmed.time, dfmed.loss.ewm(com=10).mean(), linewidth=0.75, c='#1f78b4', label='85 Dimensions')
    ax.plot(dflrg.time, dflrg.loss.ewm(com=10).mean(), linewidth=0.75, c='#b2df8a', label='125 Dimensions')
    if old != []:
        ax.plot(dfold.time, dfold.loss.ewm(com=10).mean(), linewidth=0.75, c='black', label='Unreduced Data,\nNo regularization.')
    
   # for i,f in enumerate(fails):
   #     t, b, l = f
   #     ax.plot([t-losses[0][0], t-losses[0][0]], [-100, 100], c='#fc8d62', linewidth=0.5)
    upper_lim = dfsmall.loss.append(dfmed.loss).append(dflrg.loss).append(dfold.loss).values.reshape(-1).argsort()[int(0.975*(len(dfsmall) + len(dfmed) + len(dfold.append(dflrg))))]
    upper_lim = float(dfsmall.loss.append(dfmed.loss).append(dflrg.loss).append(dfold.loss).values[upper_lim])
    lower_lim = dfsmall.append(dfmed).append(dflrg).append(dfold).loss.min() - 0.05*abs(dfsmall.append(dfmed).append(dflrg).append(dfold).loss.min())
    lower_lim = float(lower_lim)
    print(lower_lim)
    ax.set_ylim([lower_lim, upper_lim + 0.1*abs(upper_lim)])
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    if yaxis:
        ax.spines['left'].set_color("white")
        ax.spines['bottom'].set_color("white")
    else:
        ax.spines['left'].set_visible(False)
        ax.spines['bottom'].set_visible(False)
    list(map(lambda ts: ts.set_color('white'),ax.xaxis.get_majorticklines()))
    list(map(lambda ts: ts.set_color('white'),ax.yaxis.get_majorticklines()))
    ax.set_xlabel('training time in seconds')
    if yaxis:
        ax.set_ylabel(r'$\ln\mathrm{lik}(\theta | \mathbf{y}) - c$', rotation=0, horizontalalignment='right')
    if legend: plt.legend(frameon=False)
    fig.tight_layout()
    plt.savefig(filename)
    plt.show()
    print(dfsmall.time.values[-1])

In [ ]:
draw_pca_comparison(pca_50_poiss_loss, pca_85_poiss_loss, pca_125_poiss_loss, losses_large_poisson, 'figures/comparison_poiss_pca2.pdf', yaxis=True)
draw_pca_comparison(pca_50_poiss_devloss, pca_85_poiss_devloss, pca_125_poiss_devloss, dev_losses_large_poisson, 'figures/comparison_poiss_loss_pca.pdf', yaxis=True)

In [ ]:
draw_pca_comparison(pca_50_cat_loss, pca_85_cat_loss, pca_125_cat_loss, [], 'figures/comparison_cat_pca.pdf')
draw_pca_comparison(pca_50_cat_devloss, pca_85_cat_devloss, pca_125_cat_devloss, [], 'figures/comparison_cat_devloss_pca.pdf', yaxis=False)

In [ ]:
draw_pca_comparison(pca_50_poiss_devloss, pca_85_poiss_devloss, pca_125_poiss_devloss, dev_losses_med_poisson, 'figures/comparison_poiss_loss_pca.pdf', yaxis=True)


In [ ]:
draw_pca_comparison(pca_50_cat_devloss, pca_85_cat_devloss, pca_125_cat_devloss, [], 'figures/comparison_cat_loss_pca.pdf', yaxis=False)

In [ ]:
draw_comparison(dev_losses_small_cat, dev_losses_med_cat, dev_losses_large_cat, 'figures/devlosscomp_mlp_cat.pdf')

draw_comparison(dev_losses_small_poisson, dev_losses_med_poisson, dev_losses_large_poisson, 'figures/devlosscomp_mlp_poiss.pdf')

In [ ]:
# get training, dev and val sets for recurrent structure.

last_indices = reduced_data.groupby(['patient_nbr']).apply(lambda x: x.encounter_id.values.max())
last_indices = reduced_data.encounter_id.map(lambda i : i in last_indices.values)
last_val = last_indices.values & val
last_dev = last_indices.values & dev
last_train = last_indices.values & train

#last_train.shape

train_patients = reduced_data.patient_nbr[last_train]
dev_patients = reduced_data.patient_nbr[last_dev]
val_patients = reduced_data.patient_nbr[last_val]

X_ = pca.transform(prepared_data2)[:, :85]

train_batches_poisson = np.array([get_patient(pn, samples=X_, labels=y1) for pn in train_patients])
dev_batches_poisson = np.array([get_patient(pn, samples=X_, labels=y1) for pn in dev_patients])
val_batches_poisson = np.array([get_patient(pn, samples=X_, labels=y1) for pn in val_patients])

train_batches_cat = np.array([get_patient(pn, samples=X_, labels=yprob) for pn in train_patients])
dev_batches_cat = np.array([get_patient(pn, samples=X_, labels=yprob) for pn in dev_patients])
val_batches_cat = np.array([get_patient(pn, samples=X_, labels=yprob) for pn in val_patients])

In [ ]:
train_batches_cat[0][0].shape

In [ ]:
train_batches_poisson[0][0].shape

In [ ]:
def regenerate_rec():
    insrec = keras.layers.Input(shape=(None,85))
    xrec = keras.layers.GRU(units=400, return_sequences=True)(insrec)
    xrec = keras.layers.GRU(units=450, return_sequences=True)(xrec)
    xrec = keras.layers.GRU(units=400, return_sequences=True)(xrec)
    xrec = keras.layers.Dense(300)(xrec)
    xrec = keras.layers.ReLU()(xrec)
    xrec = keras.layers.Dense(200)(xrec)
    xrec = keras.layers.ReLU()(xrec)
    xrec = keras.layers.Dense(100)(xrec)
    xrec = keras.layers.ReLU()(xrec)
    xrec = keras.layers.Dense(50)(xrec)
    xrec = keras.layers.ReLU()(xrec)
    
    xpoiss = keras.layers.Dense(1)(xrec)
    xpoiss = keras.layers.ReLU()(xpoiss)
    
    xcat = keras.layers.Dense(14)(xrec)
    xcat = keras.activations.softmax(xcat)
    
    poisson_model = keras.models.Model(inputs=insrec, outputs=xpoiss)
    poisson_model.compile(loss=rec_poiss, optimizer=Adam())
    
    cat_model = keras.models.Model(inputs=insrec, outputs=xcat)
    cat_model.compile(loss=rec_cat, optimizer=Adam())
    
    return poisson_model, cat_model

In [ ]:
poisson_rec, cat_rec = regenerate_rec()

def rec_poiss(y, y_):
    return keras.losses.poisson(y[-1], y_[-1])

def rec_cat(y, y_):
    return keras.losses.categorical_crossentropy(y[-1], y_[-1])

In [ ]:
def rec_experiment(model, lossmthd, trainbatches, devbatches, maxtime):
    
    ei_patients = enumerate_cycle(trainbatches)
    loss_history_rec = []
    dev_loss_history_rec = []

    # Accumulate some stats as we go along
    #loss_history_l = []
    #dev_loss_history_l = []
    #dev_loss_history_l_fs = []

    model.compile(loss=lossmthd, optimizer=Adam())
    print('compiled model')
    while not interrupted():
        if(len(loss_history_rec) > 0 and loss_history_rec[-1][0]-loss_history_rec[0][0] >= maxtime):
            break
        (epoch,item),patient = next(ei_patients)
        #print('got patient')
        for _ in range(2):
            loss = model.train_on_batch(patient[0][np.newaxis,:,:], patient[1][np.newaxis, :, np.newaxis])
        #print('got loss')
        #print(model.predict_on_batch(X[i]))
        #input()
        #loss = model.train_on_batch(T[i], trainy[i])
        if np.isnan(loss): break
        if item % 100 == 0:
            IPython.display.clear_output(wait=True)
            print(f'loss={loss} epoch={epoch} batchnum={item}' + (f' time={loss_history_rec[-1][0]-loss_history_rec[0][0]}' if len(loss_history_rec) > 0 else ''))
            loss_history_rec.append((time.time(), epoch, loss))
            devp = devbatches[np.random.choice(np.arange(len(devbatches)))]
           # print(ydprob[sample].shape,  model.predict_on_batch(D[sample]).numpy().shape)
            devloss = tf.reduce_mean(lossmthd(devp[1], model.predict_on_batch(devp[0][np.newaxis, :, :])))
            dev_loss_history_rec.append((time.time(), epoch+batch/len(indices), devloss))
            if len(dev_loss_history_rec) > 20 and dev_loss_history_rec[-1][-1] > dev_loss_history_rec[-2][-1] and dev_loss_history_rec[-2][-1] > dev_loss_history_rec[-3][-1] and ((dev_loss_history_rec[-1][-1] > dev_loss_history_rec[-3][-1] + 0.075) or (dev_loss_history_rec[-1][-1] > min(map(lambda x: x[-1], dev_loss_history_rec)) + 0.1)):
                print("FAAIIIIL")
                dev_loss_history_l_fs.append((time.time(), epoch, devloss))
            else:
                print()
                                  
    return loss_history_rec, dev_loss_history_rec, patient

In [ ]:
#poisson_rec, _ = regenerate_rec()
#ls, dls, patient = rec_experiment(poisson_rec, rec_poiss, train_batches_poisson, dev_batches_poisson, 600)
#draw_losses(ls, dls, [], 'figures/rec_poiss_loss.pdf')
_, cat_rec = regenerate_rec()
cls, dcls, patient2 = rec_experiment(cat_rec, rec_cat, train_batches_cat, dev_batches_cat, 600)
draw_losses(cls, dcls, [], 'figures/rec_cat_loss.pdf')

In [ ]:
draw_losses(ls, dls, [], 'figures/rec_poisson_loss.pdf', comp=25)
draw_losses(cls, dcls, [], 'figures/rec_cat_loss.pdf', comp=25)

In [ ]:
train_batches_cat[345][0][np.newaxis, :, :]

In [ ]:
ys_poiss = []

for b in tqdm(val_batches_poisson[:, 0]):
    
    y_ = poisson_rec.predict_on_batch(b[np.newaxis, :, :])
    ys_poiss.append(y_.numpy().reshape(-1)[-1])

In [ ]:
recvalev = np.array([list(map(lambda x: x[-1], val_batches_poisson[:, 1])), ys_])
from scipy.stats import pearsonr
pearsonr(recvalev[0], recvalev[1])

In [ ]:
def draw_cat_resps(model, valx, valy, file_name, yps_=None):
    if(type(yps_) == 'NoneType'):
        yps_ = model.predict_on_batch(valx).numpy()
    
    fig, ax = plt.subplots()
    print(yval == 9)
    for _y in np.arange(1,15):
        inds = valy == _y
        print(yps_)
        ys_rv = yps_[inds]
       # print(ys_rv.shape)
        responsibilities = ys_rv.mean(axis=0)
       # print(responsibilities)
        for i in np.arange(1,15):
            ax.fill([_y-0.4875, _y-0.4875, _y+0.4875, _y+0.4875], [i-0.4875, i+0.4875, i+0.4875, i-0.4875], c='black', alpha=responsibilities[i-1])
        

In [ ]:
ys_poiss

In [ ]:
ys_ = []
for b in tqdm(val_batches_cat[:, 0]):
    y_ = cat_rec.predict_on_batch(b[np.newaxis, :, :])
    ys_.append(y_.numpy()[0][-1])

In [ ]:
val_pca_50_cat

In [ ]:
recyval = np.array(list(map(lambda x: x[-1], )))

In [ ]:
recyval = []
for l in val_batches_poisson[:, 1]:
    recyval.extend(l)

In [ ]:
ys_

In [ ]:
draw_comparison(pca_85_cat_loss, pca_85_cat_loss, cls, '', comp=100)

In [ ]:
1-reduced_data.patient_nbr.value_counts().value_counts()[1]/len(reduced_data)

In [ ]:
val_mlp_lrg_cat = large_model_cat.predict_on_batch(V).numpy().argmax(axis=1) + 1
val_mlp_med_cat = med_model_cat.predict_on_batch(V).numpy().argmax(axis=1) + 1
val_mlp_sml_cat = small_model_cat.predict_on_batch(V).numpy().argmax(axis=1) + 1

val_mlp_lrg_poisson = large_model_poisson.predict_on_batch(V).numpy().reshape(-1)
val_mlp_med_poisson = med_model_poisson.predict_on_batch(V).numpy().reshape(-1)
val_mlp_sml_poisson = small_model_poisson.predict_on_batch(V).numpy().reshape(-1)

val_pca_50_cat =  pca_cat_50.predict_on_batch(V_50).numpy().argmax(axis=1) + 1
val_pca_85_cat =  pca_cat_85.predict_on_batch(V_85).numpy().argmax(axis=1) + 1
val_pca_125_cat = pca_cat_125.predict_on_batch(V_125).numpy().argmax(axis=1) + 1

val_pca_50_poisson = pca_poiss_50.predict_on_batch(V_50).numpy().reshape(-1)
val_pca_85_poisson = pca_poiss_85.predict_on_batch(V_85).numpy().reshape(-1)
val_pca_125_poisson = pca_poiss_125.predict_on_batch(V_125).numpy().reshape(-1)

val_rec_cat = []
val_rec_cat_prob = []
yval_p = []
for i, (b,y) in enumerate(tqdm(val_batches_cat)):
    y_ = cat_rec.predict_on_batch(b[np.newaxis, :, :]).numpy().argmax(axis=2).reshape(-1)
    val_rec_cat.append(y_[-1])
    yval_p.append(y.reshape(-1)[-1])
    val_rec_cat_prob.append(cat_rec.predict_on_batch(b[np.newaxis, :, :]).numpy()[-1])
    
print(y)
val_rec_poiss = []
yval_c = []
for i, (b,y) in tqdm(enumerate(val_batches_poisson)):
    y_ = poisson_rec.predict_on_batch(b[np.newaxis, :, :]).numpy().reshape(-1)
    val_rec_poiss.append(y_[-1])
    yval_c.append(y.reshape(-1)[-1])

In [ ]:
print('MLP:')
print('', 'Cat')
print('', '', 'Large', pearsonr(val_mlp_lrg_cat, yval))
print('', '', 'Mediu', pearsonr(val_mlp_med_cat, yval))
print('', '', 'Small', pearsonr(val_mlp_sml_cat, yval))
print('', 'Poiss')
print('', '', 'Large', pearsonr(val_mlp_lrg_poisson, yval))
print('', '', 'Mediu', pearsonr(val_mlp_med_poisson, yval))
print('', '', 'Small', pearsonr(val_mlp_sml_poisson, yval))
print('PCA:')
print('', 'Cat')
print('', '', '050', pearsonr(val_pca_50_cat, yval))
print('', '', '085', pearsonr(val_pca_85_cat, yval))
print('', '', '125', pearsonr(val_pca_125_cat, yval))
print('', 'Poiss')
print('', '', '050', pearsonr(val_pca_50_poisson, yval))
print('', '', '085', pearsonr(val_pca_85_poisson, yval))
print('', '', '125', pearsonr(val_pca_125_poisson, yval))
print('Rec:')
print('', 'X-Ent', pearsonr(val_rec_cat, yval_c))
print('', 'Poiss', pearsonr(val_rec_poiss, yval_c))

In [ ]:
yps_ = small_model_poisson.predict_on_batch(V).numpy()
#ys_ = (yps_.argmax(axis=1) + 1)

fig, ax = plt.subplots()
for y, y_ in tqdm(zip(yval, yps_)):
    ax.plot([y-0.5, y+0.5], [y_, y_], linewidth=0.5, c='black', alpha=0.1)   
ax.set_xlim(0.5, 14.5)
ax.set_ylim(0.5, 14.5)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_color('white')
ax.spines['bottom'].set_color('white')
list(map(lambda ts: ts.set_color('white'),ax.xaxis.get_majorticklines()))
list(map(lambda ts: ts.set_color('white'),ax.yaxis.get_majorticklines()))
ax.set_xlabel('True Duration')
ax.set_ylabel('Predicted Duration')
ax.set_aspect('equal')
print('saving now')
plt.savefig('figures/small_dev_dis.pdf')
print('saved')
plt.show()

In [ ]:
yps_ = pca_cat_125.predict_on_batch(V_125).numpy()
#ys_ = (yps_.argmax(axis=1) + 1)

fig, ax = plt.subplots()
for _y in np.arange(1,15):
    inds = yval == _y
    ys_rv = yps_[inds]
    responsibilities = ys_rv.mean(axis=0)
    for i in np.arange(1,15):
        ax.fill([_y-0.4875, _y-0.4875, _y+0.4875, _y+0.4875], [i-0.4875, i+0.4875, i+0.4875, i-0.4875], c='black', alpha=responsibilities[i-1])
        
ax.set_xlim(0.5, 14.5)
ax.set_ylim(0.5, 14.5)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.spines['left'].set_color('white')
ax.spines['bottom'].set_color('white')
list(map(lambda ts: ts.set_color('white'),ax.xaxis.get_majorticklines()))
list(map(lambda ts: ts.set_color('white'),ax.yaxis.get_majorticklines()))
ax.set_xlabel('True Duration')
ax.set_ylabel('Predicted Duration')
ax.set_aspect('equal')
plt.savefig('figures/pca125_val_resps.pdf')
plt.show()

In [ ]:
med_model_cat.predict_on_batch(V).numpy()

In [ ]:
cat_rec.predict_on_batch(train_batches_cat[1352, 0][np.newaxis, :, :]).numpy()

In [ ]:
responsibilities

In [ ]:
poisson_rec.predict_on_batch(patient[0][np.newaxis, :, :]).numpy().reshape(-1)

In [ ]:
keras.losses.poisson(np.array([4.]), np.array([0.04232]))

In [ ]:
yval_p

In [ ]:
for _ in tqdm(range(5)):
    pass #poisson_rec.train_on_batch(train_batches_poisson[129, 0][np.newaxis, :, :], train)
    
print(poisson_rec.predict_on_batch(train_batches_poisson[4152,0][np.newaxis, :, :]).numpy(), train_batches_poisson[4152, 1])
print(poisson_rec.predict_on_batch(train_batches_poisson[1146,0][np.newaxis, :, :]).numpy(), train_batches_poisson[1146, 1])
print(poisson_rec.predict_on_batch(train_batches_poisson[3123,0][np.newaxis, :, :]).numpy())
print(poisson_rec.predict_on_batch(patient[0][np.newaxis, :, :]))

patient[1][:, np.newaxis]